In [0]:

## Libraries Prequisities
!pip install scikit-learn
!pip install numpy==1.16.1

     |████████████████████████████████| 17.3MB 200kB/s 
ERROR: tensorflow-model-optimization 0.2.1 requires enum34~=1.1, which is not installed.
ERROR: tensorflow-federated 0.12.0 has requirement tensorflow~=2.1.0, but you'll have tensorflow 1.15.0 which is incompatible.
ERROR: tensorflow-federated 0.12.0 has requirement tensorflow-addons~=0.7.0, but you'll have tensorflow-addons 0.8.3 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: numpy 1.18.2
    Uninstalling numpy-1.18.2:
      Successfully uninstalled numpy-1.18.2


In [1]:
import numpy as np
import random
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.metrics.classification import accuracy_score, recall_score, f1_score
import scipy.stats as st

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.classification module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
%tensorflow_version 2.x
import tensorflow as tf
print(tf.__version__)
print(tf.test.gpu_device_name())
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

2.2.0-rc2
/device:GPU:0
Num GPUs Available:  1


In [0]:
from tensorflow.keras.callbacks import Callback

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
np.random.seed(12227)

In [0]:
n_ep = 200
loss_ = 0.2
bs = 1000

In [0]:
def custom_stopping(value=0.5, verbose=0):
    early = EarlyStoppingByLossVal(monitor='val_loss', value=value, verbose=verbose)
    return early

class EarlyStoppingByLossVal(Callback):
    def __init__(self, monitor='val_acc', value=0.95, verbose=0):
        super(Callback, self).__init__()
        self.monitor = monitor
        self.value = value
        self.verbose = verbose

    def on_epoch_end(self, epoch, logs={}):
        current = logs.get(self.monitor)
        # if current is None:
        # warnings.warn("Early stopping requires %s available!" % self.monitor, RuntimeWarning)

        if current < self.value:
            if self.verbose > 0:
                print("Epoch %05d: early stopping THR" % epoch)
            self.model.stop_training = True

In [0]:
def DataPreparation(data_input_file,data_target_file,test_file,test_target_file):
  print('Panwar  et al. 2017 {}'.format(data_input_file))
  # data = np.load(data_input_file)
  # X = data['X']
  X=np.load(data_input_file)
  # X = X[:, 0, :, :]
  Y = np.load(data_target_file)
  # folds = data['folds']
  # classes_number = Y.shape[1]
  # Y = np.argmax(Y, axis=1)
  X_test=np.load(test_file)
  Y_test=np.load(test_target_file)
  return X.f.arr_0,Y.f.arr_0, X_test.f.arr_0,Y_test.f.arr_0

In [0]:
def ReportAccuracies(avg_acc, avg_recall,avg_f1):
  ic_acc = st.t.interval(0.9, len(avg_acc) - 1, loc=np.mean(avg_acc), scale=st.sem(avg_acc))
  ic_recall = st.t.interval(0.9, len(avg_recall) - 1, loc=np.mean(avg_recall), scale=st.sem(avg_recall))
  ic_f1 = st.t.interval(0.9, len(avg_f1) - 1, loc=np.mean(avg_f1), scale=st.sem(avg_f1))
  print('Mean Accuracy[{:.4f}] IC [{:.4f}, {:.4f}]'.format(np.mean(avg_acc), ic_acc[0], ic_acc[1]))
  print('Mean Recall[{:.4f}] IC [{:.4f}, {:.4f}]'.format(np.mean(avg_recall), ic_recall[0], ic_recall[1]))
  print('Mean F1[{:.4f}] IC [{:.4f}, {:.4f}]'.format(np.mean(avg_f1), ic_f1[0], ic_f1[1]))

In [0]:
def RunPanware(data_input_file,data_target_file,test_file,test_target_file):
  X,Y,X_test,Y_test= DataPreparation(data_input_file,data_target_file,test_file,test_target_file)
  # idx = [3, 5, 8, 11, 14, 17, 20] #For MHEALTH
  # idx = [1, 4, 7, 10, 13, 14, 17, 20, 23, 26, 27, 30, 33, 36]#For PAMAP2P
  # idx = [val for val in idx for _ in (0, 1)] #Vertical Kernel-1
  # X = zero_padding_MHEALTH(X)
  avg_acc, avg_recall,avg_f1= Train(X,Y,X_test,Y_test)
  ReportAccuracies(avg_acc, avg_recall,avg_f1 )

In [0]:
def build_model_1(row,col,num_classes,inp):

  #Architecture1 from paper
  # activation = 'relu'

  # H = tf.keras.layers.Conv2D(filters=5, kernel_size=(9, 3), padding = 'same')(inp)
  # H = tf.keras.layers.Conv2D(filters=5, kernel_size=(5, 3), padding = 'same')(H)
  # H = tf.keras.layers.MaxPooling2D(pool_size=(2, 1))(H)


  # H = tf.keras.layers.Activation('relu')(H)

  # H = tf.keras.layers.Flatten()(H)
  # H = tf.keras.layers.Dense(num_classes)(H)
  # H = tf.keras.layers.Activation('softmax')(H)

  # model = Model([inp], H)

 
  layers = [
        tf.keras.layers.Conv2D(filters=5,kernel_size=(9, 3)),
        
        tf.keras.layers.Conv2D(5,(5, 3)),
        tf.keras.layers.MaxPool2D((2, 1)),
        tf.keras.layers.Activation('relu'),      

       

       
        tf.keras.layers.Flatten(),
       
        tf.keras.layers.Dense(num_classes, activation = 'softmax')
    ]
    
  model = tf.keras.Sequential(layers)
  return model

In [0]:
def build_model_2(row,col,num_classes):
  tf.initializers.Orthogonal(gain=1.0, seed=None)
  layers = [
        tf.keras.layers.Conv2D(5,(9, 3),padding='same'),
        tf.keras.layers.Conv2D(5,(7, 3),padding='same'),
        tf.keras.layers.Conv2D(5,(5, 3),padding='same'),
        tf.keras.layers.MaxPool2D((2, 1)),
        tf.keras.layers.Activation('relu'),      

       

       
        tf.keras.layers.Flatten(),
       
        tf.keras.layers.Dense(num_classes, activation = 'softmax')
    ]
    
  model = tf.keras.Sequential(layers)
  return model

In [0]:
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import train_test_split

In [0]:
def Train(X,y,X_test,Y_test):
  avg_acc = []
  avg_recall = []
  avg_f1 = []
  n_class = y.shape[1]
  _, img_rows, img_cols = X.shape
  X=X.reshape(X.shape[0],img_rows,img_cols,1)
  _,img_rows_, img_cols_ = X_test.shape
  X_test=X_test.reshape(X_test.shape[0],img_rows_,img_cols_,1)
  print(n_class)
  # y=y.reshape(np.shape(y)[0],1)


  print(X.shape)
  print(y.shape)
  kf = KFold(n_splits=5, random_state=1442, shuffle=True)
  kf.get_n_splits(X)
  n_fold=1
  for train_ids_indx, valid_ids_indx in kf.split(X):
    # train_idx = folds[i][0]
    # test_idx = folds[i][1]
    X_train = X[train_ids_indx]
    X_valid = X[valid_ids_indx]
    y_train= y[train_ids_indx]
    y_valid=y[valid_ids_indx]
    print(X_train.shape)
    model=build_model_1(img_rows,img_cols,n_class,tf.keras.layers.Input((1, img_rows, img_cols)))
    # print(model.summary)
    model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='SGD')
    model.fit(X_train, y_train, batch_size=bs, epochs=n_ep,
                  verbose=0, callbacks=[custom_stopping(value=loss_, verbose=1)],validation_data=(X_valid,y_valid))
    y_pred = model.predict(X_test)
    y_pred = np.argmax(y_pred, axis=1)
    y_true = np.argmax(Y_test, axis=1)
    acc_fold = accuracy_score(y_true, y_pred)
    avg_acc.append(acc_fold)
    recall_fold = recall_score(y_true, y_pred, average='macro')
    avg_recall.append(recall_fold)
    f1_fold = f1_score(y_true, y_pred, average='macro')
    avg_f1.append(f1_fold)

    print('Accuracy[{:.4f}] Recall[{:.4f}] F1[{:.4f}] at fold[{}]'.format(acc_fold, recall_fold, f1_fold ,n_fold))
    print('________________________________________________________________')
    n_fold+=1
    
  return avg_acc, avg_recall,avg_f1

In [0]:
tf.keras.backend.set_image_data_format('channels_first')

In [0]:
X=np.load('/content/drive/My Drive/Computer_vision_project/data/UCI_Heterogeneity /UCI_HHAR_Y_2.npz')

In [0]:
sorted(X.files)

['arr_0']

In [0]:
X.f.arr_0

array([4., 4., 4., ..., 4., 4., 4.])

In [0]:
X.f.arr_0.shape

(102050,)

In [16]:
RunPanware('/content/drive/My Drive/Computer_vision_project/data/Opportunity/Opportunity_train_X.npz','/content/drive/My Drive/Computer_vision_project/data/Opportunity/Opportunity_train_y.npz','/content/drive/My Drive/Computer_vision_project/data/Opportunity/Opportunity_test_X.npz','/content/drive/My Drive/Computer_vision_project/data/Opportunity/Opportunity_test_Y.npz')

Panwar  et al. 2017 /content/drive/My Drive/Computer_vision_project/data/Opportunity/Opportunity_train_X.npz
5
(46495, 24, 113, 1)
(46495, 5)
(37196, 24, 113, 1)
Accuracy[0.8340] Recall[0.8464] F1[0.8486] at fold[1]
________________________________________________________________
(37196, 24, 113, 1)
Accuracy[0.8456] Recall[0.8690] F1[0.8671] at fold[2]
________________________________________________________________
(37196, 24, 113, 1)
Accuracy[0.8487] Recall[0.8629] F1[0.8647] at fold[3]
________________________________________________________________
(37196, 24, 113, 1)
Accuracy[0.8406] Recall[0.8567] F1[0.8575] at fold[4]
________________________________________________________________
(37196, 24, 113, 1)
Accuracy[0.8423] Recall[0.8560] F1[0.8594] at fold[5]
________________________________________________________________
Mean Accuracy[0.8422] IC [0.8370, 0.8475]
Mean Recall[0.8582] IC [0.8501, 0.8663]
Mean F1[0.8594] IC [0.8526, 0.8663]
